In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from trading_strategy.data_fetcher import DataFetcher
from trading_strategy.indicator_calculator import IndicatorCalculator
from trading_strategy.signal_processor import SignalProcessor
from trading_strategy.backtester import Backtester
from config.config import API_KEY, SECRET_KEY, BASE_URL, ASSET_CLASS, SYMBOL, TIMEFRAME_PIVOT_POINT, START_DATE, END_DATE , RSI_PERIOD, FAST_PERIOD, SLOW_PERIOD, MIDDLE_PERIOD, MIN_ANGLE, MAX_ANGLE


ModuleNotFoundError: No module named 'trading_strategy'

In [ ]:
# Alpaca
API_KEY = API_KEY
SECRET_KEY = SECRET_KEY
BASE_URL = BASE_URL

In [ ]:
symbol = SYMBOL
asset_class = ASSET_CLASS   
start_date = START_DATE
end_date = END_DATE
timeframe_pivot_point = TIMEFRAME_PIVOT_POINT
rsi_period = RSI_PERIOD
fast_period = FAST_PERIOD
slow_period = SLOW_PERIOD
middle_period = MIDDLE_PERIOD
min_angle = MIN_ANGLE
max_angle = MAX_ANGLE

In [ ]:
# Backtest the strategy to find the best performance across timeframes and pivot points

pivot_points = [['pivot_support_1', 'pivot_resistance_1'], ['pivot_support_2', 'pivot_resistance_2'], ['pivot_support_3', 'pivot_resistance_3']]
signal_timeframes = [
                     ['signal_15min'],
                     ['signal_30min'],
                     ['signal_1H'],
                     ['signal_4H'],
                     ['signal_1D'],
                     ['signal_15min', 'signal_30min'],
                     ['signal_15min', 'signal_1H'],
                     ['signal_15min', 'signal_4H'],
                     ['signal_30min', 'signal_1H'],
                     ['signal_30min', 'signal_4H'],
                     ['signal_1H', 'signal_4H'],
                     ['signal_1H', 'signal_1D'],
                     ['signal_4H', 'signal_1D'],
                     ['signal_15min', 'signal_30min', 'signal_1H'],
                     ['signal_15min', 'signal_30min', 'signal_4H'],
                     ['signal_15min', 'signal_30min', 'signal_1D'],
                     ['signal_15min', 'signal_1H', 'signal_4H'],
                     ['signal_15min', 'signal_1H', 'signal_1D'],
                     ['signal_30min', 'signal_1H', 'signal_4H'],
                     ['signal_30min', 'signal_1H', 'signal_1D'],
                     ['signal_1H', 'signal_4H', 'signal_1D'],
                     ['signal_15min', 'signal_30min', 'signal_1H', 'signal_4H'],
                     ['signal_15min', 'signal_30min', 'signal_1H', 'signal_1D'],
                     ['signal_30min', 'signal_1H', 'signal_4H', 'signal_1D'],
                     ['signal_15min', 'signal_30min', 'signal_1H', 'signal_4H', 'signal_1D'],
                     ]
df_metrics = pd.DataFrame(columns=['signal_timeframe', 'pivot_point', 'cumulative_return', 'volatility', 'annualized_return', 'sharpe_ratio', 'number_of_trades', 'average_return_per_trade', 'max_drawdown', 'average_holding_period', 'winning_trades_ratio', 'buy_trades_ratio'])
datafetcher = DataFetcher(API_KEY, SECRET_KEY, BASE_URL)
signal_processor = SignalProcessor(API_KEY, SECRET_KEY, BASE_URL)

for i in pivot_points:

    pivot_support_level = i[0]
    pivot_resistance_level = i[1]

    # Pivot point for stop loss and take profit
    df = datafetcher.get_data(symbol, timeframe_pivot_point, start_date, end_date, asset_class)
    df_pivot_point = pd.DataFrame()
    df_pivot_point['pivot_support_0'], df_pivot_point['pivot_support_1'], df_pivot_point['pivot_support_2'], df_pivot_point['pivot_support_3'], df_pivot_point['pivot_resistance_0'], df_pivot_point['pivot_resistance_1'], df_pivot_point['pivot_resistance_2'], df_pivot_point['pivot_resistance_3'] = IndicatorCalculator.calculate_pivot_point(df)

    # Multitimeframe dataframe to find trading signal
    df_15m = signal_processor.generate_signals_dataframe(symbol, '15min', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_30m = signal_processor.generate_signals_dataframe(symbol, '30min', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_1h = signal_processor.generate_signals_dataframe(symbol, '1H', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_4h = signal_processor.generate_signals_dataframe(symbol, '4H', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_1d = signal_processor.generate_signals_dataframe(symbol, '1D', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)

    df = pd.merge(df_15m, df_30m,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_1h,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_4h,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_1d,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_pivot_point, left_index = True, right_index=True, how='outer') 
    df = df.fillna(method='ffill')
    df.dropna(inplace=True)

    # Create a dataframe of close price of the smallest timeframe in the multitimeframe dataset
    shortest_timeframe = 'close_15min'
    df_price = df_15m.loc[:, shortest_timeframe]
    df_price.name = 'close_price'

    ###

    # Define the timeframe useful for the signal
    for i in signal_timeframes:
        
        signal_timeframe = i

        try:
            # Create a DataFrame of buy signals
            buy_condition = (df[signal_timeframe] == 3).all(axis=1)
            df_buy = df[buy_condition]
            
            if len(df_buy) > 0:
                df_buy = df_buy.loc[:, [shortest_timeframe, f'{pivot_resistance_level}', f'{pivot_support_level}']]
                df_buy = df_buy.rename(columns={
                    shortest_timeframe: 'entry_price',
                    f'{pivot_support_level}': 'stop_loss',
                    f'{pivot_resistance_level}': 'take_profit'
                })
                df_buy = Backtester.find_exit_price_and_time_of_signal(df_price, df_buy, 'buy')
                df_buy['return'] = df_buy['exit_price'] / df_buy['entry_price'] - 1
                df_buy['action'] = 'buy'
            else:
                print(f"No buy signals found for: {signal_timeframe}, {pivot_support_level}, {pivot_resistance_level}")
                df_buy = pd.DataFrame()  # Ensure df_buy is an empty DataFrame if no buy signals exist

        except KeyError as e:
            print(f"KeyError encountered in buy signal processing: {e}")
        except Exception as e:
            print(f"An error occurred while processing buy signals: {e}")


        try:
            # Create a DataFrame of sell signals
            sell_condition = (df[signal_timeframe] == -3).all(axis=1)
            df_sell = df[sell_condition]
            
            if len(df_sell) > 0:
                df_sell = df_sell.loc[:, [shortest_timeframe, f'{pivot_resistance_level}', f'{pivot_support_level}']]
                df_sell = df_sell.rename(columns={
                    shortest_timeframe: 'entry_price',
                    f'{pivot_resistance_level}': 'stop_loss',
                    f'{pivot_support_level}': 'take_profit'
                })
                df_sell = Backtester.find_exit_price_and_time_of_signal(df_price, df_sell, 'sell')
                df_sell['return'] = -(df_sell['exit_price'] / df_sell['entry_price'] - 1)
                df_sell['action'] = 'sell'
            else:
                print(f"No sell signals found for: {signal_timeframe}, {pivot_support_level}, {pivot_resistance_level}")
                df_sell = pd.DataFrame()  

        except KeyError as e:
            print(f"KeyError encountered in sell signal processing: {e}")
        except Exception as e:
            print(f"An error occurred while processing sell signals: {e}")

        # Combine buy and sell signals into one DataFrame
        try:
            df_buy_sell = pd.concat([df_buy, df_sell])
            if len(df_buy_sell) > 0:
                # Only one position at a time, ensure the first position is closed before opening another
                df_buy_sell.sort_index(ascending=True, inplace=True)
                exit = df_buy_sell.iloc[0, 3] #3 for exit price column
                index = [0]
                
                for i in range(1, len(df_buy_sell)):
                    if df_buy_sell.index[i] > exit:
                        index.append(i)
                        exit = df_buy_sell.iloc[i, 3]
                
                df_buy_sell = df_buy_sell.iloc[index, :]

                ###

                # Compute the metrics
                cumulative_return, volatility, annualized_return, sharpe_ratio, number_of_trades, average_return_per_trade, max_drawdown, average_holding_period, winning_trades_ratio, buy_trades_ratio = Backtester.calculate_risk_performance_mectrics(df_buy_sell)

                # Keep the metrics in the dataframe
                new_iteration = {'signal_timeframe':signal_timeframe, 
                                'pivot_point': [pivot_support_level, pivot_resistance_level], 
                                'cumulative_return': cumulative_return, 
                                'volatility': volatility, 
                                'annualized_return': annualized_return, 
                                'sharpe_ratio': sharpe_ratio, 
                                'number_of_trades': number_of_trades, 
                                'average_return_per_trade': average_return_per_trade, 
                                'max_drawdown': max_drawdown, 
                                'average_holding_period': average_holding_period,
                                'winning_trades_ratio': winning_trades_ratio, 
                                'buy_trades_ratio': buy_trades_ratio}
                df_metrics = pd.concat([df_metrics, pd.DataFrame([new_iteration])], ignore_index=True)

            else:
                new_iteration = {'signal_timeframe':signal_timeframe, 
                                'pivot_point': [pivot_support_level, pivot_resistance_level], 
                                'cumulative_return': 0, 
                                'volatility': 0, 
                                'annualized_return': 0, 
                                'sharpe_ratio': 0, 
                                'number_of_trades': 0, 
                                'average_return_per_trade': 0, 
                                'max_drawdown': 0, 
                                'average_holding_period': 0,
                                'winning_trades_ratio': 0, 
                                'buy_trades_ratio': 0}
                df_metrics = pd.concat([df_metrics, pd.DataFrame([new_iteration])], ignore_index=True)
                print("No buy or sell signals found. df_buy_sell is empty.")

        except Exception as e:
            print(f"An error occurred while combining and processing df_buy and df_sell: {e}")

        print('Finished for :', signal_timeframe)

In [ ]:
df_metrics.sort_values(by='cumulative_return', ascending = False)

In [ ]:
# Analyse the trades return and cumulative return of the best parametrized strategy
pivot_points = [['pivot_support_1', 'pivot_resistance_1']]
signal_timeframes = [['signal_4H']]


for i in pivot_points:

    pivot_support_level = i[0]
    pivot_resistance_level = i[1]

    # Pivot point for stop loss and take profit
    datafetcher = DataFetcher(API_KEY, SECRET_KEY, BASE_URL)
    df = datafetcher.get_data( symbol, timeframe_pivot_point, start_date, end_date, asset_class)
    df_pivot_point = pd.DataFrame()
    df_pivot_point['pivot_support_0'], df_pivot_point['pivot_support_1'], df_pivot_point['pivot_support_2'], df_pivot_point['pivot_support_3'], df_pivot_point['pivot_resistance_0'], df_pivot_point['pivot_resistance_1'], df_pivot_point['pivot_resistance_2'], df_pivot_point['pivot_resistance_3'] = IndicatorCalculator.calculate_pivot_point(df)

    # Multitimeframe dataframe to find trading signal
    signal_processor = SignalProcessor(API_KEY, SECRET_KEY, BASE_URL)
    df_15m = signal_processor.generate_signals_dataframe(symbol, '15min', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_30m = signal_processor.generate_signals_dataframe(symbol, '30min', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_1h = signal_processor.generate_signals_dataframe(symbol, '1H', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_4h = signal_processor.generate_signals_dataframe(symbol, '4H', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)
    df_1d = signal_processor.generate_signals_dataframe(symbol, '1D', start_date, end_date, rsi_period, fast_period, slow_period, middle_period, min_angle, max_angle, asset_class)

    df = pd.merge(df_15m, df_30m,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_1h,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_4h,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_1d,  left_index=True, right_index=True, how='outer') 
    df = pd.merge(df, df_pivot_point, left_index = True, right_index=True, how='outer') 
    df = df.fillna(method='ffill')
    df.dropna(inplace=True)

    # Create a dataframe of close price of the smallest timeframe in the multitimeframe dataset
    shortest_timeframe = 'close_15min'
    df_price = df_15m.loc[:, shortest_timeframe]
    df_price.name = 'close_price'

    ###

    # Define a timeframe to find signal
    for i in signal_timeframes:
        
        signal_timeframe = i

        try:
            # Create a DataFrame of buy signals
            buy_condition = (df[signal_timeframe] == 3).all(axis=1)
            df_buy = df[buy_condition]
            
            if len(df_buy) > 0:
                df_buy = df_buy.loc[:, [shortest_timeframe, f'{pivot_resistance_level}', f'{pivot_support_level}']]
                df_buy = df_buy.rename(columns={
                    shortest_timeframe: 'entry_price',
                    f'{pivot_support_level}': 'stop_loss',
                    f'{pivot_resistance_level}': 'take_profit'
                })
                df_buy = Backtester.find_exit_price_and_time_of_signal(df_price, df_buy, 'buy')
                df_buy['return'] = df_buy['exit_price'] / df_buy['entry_price'] - 1
                df_buy['action'] = 'buy'
            else:
                print(f"No buy signals found for: {signal_timeframe}, {pivot_support_level}, {pivot_resistance_level}")
                df_buy = pd.DataFrame()  # Ensure df_buy is an empty DataFrame if no buy signals exist

        except KeyError as e:
            print(f"KeyError encountered in buy signal processing: {e}")
        except Exception as e:
            print(f"An error occurred while processing buy signals: {e}")


        try:
            # Create a DataFrame of sell signals
            sell_condition = (df[signal_timeframe] == -3).all(axis=1)
            df_sell = df[sell_condition]
            
            if len(df_sell) > 0:
                df_sell = df_sell.loc[:, [shortest_timeframe, f'{pivot_resistance_level}', f'{pivot_support_level}']]
                df_sell = df_sell.rename(columns={
                    shortest_timeframe: 'entry_price',
                    f'{pivot_resistance_level}': 'stop_loss',
                    f'{pivot_support_level}': 'take_profit'
                })
                df_sell = Backtester.find_exit_price_and_time_of_signal(df_price, df_sell, 'sell')
                df_sell['return'] = -(df_sell['exit_price'] / df_sell['entry_price'] - 1)
                df_sell['action'] = 'sell'
            else:
                print(f"No sell signals found for: {signal_timeframe}, {pivot_support_level}, {pivot_resistance_level}")
                df_sell = pd.DataFrame()  

        except KeyError as e:
            print(f"KeyError encountered in sell signal processing: {e}")
        except Exception as e:
            print(f"An error occurred while processing sell signals: {e}")

        # Combine buy and sell signals into one DataFrame
        try:
            df_buy_sell = pd.concat([df_buy, df_sell])
            if len(df_buy_sell) > 0:
                # Only one position at a time, ensure the first position is closed before opening another
                df_buy_sell.sort_index(ascending=True, inplace=True)
                exit = df_buy_sell.iloc[0, 3] #3 for exit price column
                index = [0]
                
                for i in range(1, len(df_buy_sell)):
                    if df_buy_sell.index[i] > exit:
                        index.append(i)
                        exit = df_buy_sell.iloc[i, 3]
                
                df_buy_sell = df_buy_sell.iloc[index, :]

                # Compute the metrics
                cumulative_return, volatility, annualized_return, sharpe_ratio, number_of_trades, average_return_per_trade, max_drawdown, average_holding_period, winning_trades_ratio, buy_trades_ratio = Backtester.calculate_risk_performance_mectrics(df_buy_sell)
                print(f"Total Return: {cumulative_return:.2%}")
                print(f"Volatility (Annualized): {volatility:.2%}")
                print(f"Annualized Return: {annualized_return:.2%}")
                print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
                print(f"Number of Trades: {number_of_trades}")
                print(f"Average Return per Trade: {average_return_per_trade:.2%}")
                print(f"Max Drawdown: {max_drawdown:.2%}")
                print(f"Average Holding Period (days): {average_holding_period:.2f}")
                print(f"Ratio of Winning Trades: {winning_trades_ratio:.2%}")
                print(f"Ratio of Buy Trades: {buy_trades_ratio:.2%}")

                # Plotting the return and cumulative return
                plt.figure(figsize=(14, 6))
                # Plot trade returns
                plt.subplot(2, 1, 1)
                plt.plot(df_buy_sell.index, df_buy_sell['return'], label='Return', color='blue')
                plt.title('Trades Returns')
                plt.xlabel('Date')
                plt.ylabel('Return')
                plt.grid(True)
                plt.legend()
                # Plot cumulative return
                plt.subplot(2, 1, 2)
                plt.plot(df_buy_sell.index, (df_buy_sell['return'] +1).cumprod() -1, label='Cumulative Return', color='green')
                plt.title('Cumulative Return')
                plt.xlabel('Date')
                plt.ylabel('Cumulative Return')
                plt.grid(True)
                plt.legend()
                # Show the plot
                plt.tight_layout()
                plt.show()

        except Exception as e:
            print(f"An error occurred while combining and processing df_buy and df_sell: {e}")

        print('Finished for :', signal_timeframe)